In [3]:
import pandas as pd
import numpy as np

# 1. Attendance Data

Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [21]:
attendance_df = pd.read_csv('untidy-data/attendance.csv')
attendance_df.head()

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [22]:
attendance_melt = attendance_df.melt(id_vars=['Unnamed: 0'])
attendance_melt.head()

,Unnamed: 0,variable,value
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T


In [23]:
mapping = {'A': 1, 'H': .5, 'T': .1, 'P': 0}

# Replace values in the 'column_name' column using the mapping
attendance_melt['value'] = attendance_melt['value'].map(mapping)

In [24]:
attendance_melt.head()

,Unnamed: 0,variable,value
0,Sally,2018-01-01,0.0
1,Jane,2018-01-01,1.0
2,Billy,2018-01-01,1.0
3,John,2018-01-01,0.0
4,Sally,2018-01-02,0.1


In [26]:
1-attendance_melt.groupby('Unnamed: 0').mean()

/var/folders/qh/kjmdb2zx3n3b2ng_n4q3cyn00000gn/T/ipykernel_45846/1960734550.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  1-attendance_melt.groupby('Unnamed: 0').mean()


,value
Unnamed: 0,
Billy,0.5250
Jane,0.6875
John,0.9125
Sally,0.7625


You should end up with something like this:

    `name
    Billy    0.5250
    Jane     0.6875
    John     0.9125
    Sally    0.7625
    Name: grade, dtype: float64`


# 2. Coffee Levels

* a. Read the coffee_levels table.

In [27]:
cof_df = pd.read_csv('untidy-data/coffee_levels.csv')
cof_df.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


* b. Transform the data so that each carafe is in it's own column.

In [29]:
cof_df.pivot_table(values='coffee_amount', index='hour', columns='coffee_carafe')

coffee_carafe,x,y,z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947
13,0.310711,0.997464,0.398520
14,0.507288,0.058361,0.864464
15,0.215043,0.144644,0.436364
16,0.183891,0.544676,0.280621


* c. Is this the best shape for the data?

In [ ]:
Yes

# 3. Cake Recipes

* a. Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [31]:
cake_df = pd.read_csv('untidy-data/cake_recipes.csv')
cake_df.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


* b. Tidy the data as necessary.

In [32]:
cake_melt = cake_df.melt(id_vars=['recipe:position'])
cake_melt.head()

,recipe:position,variable,value
0,a:bottom,225,61.738655
1,a:top,225,51.709751
2,b:bottom,225,57.095320
3,b:top,225,82.455004
4,c:bottom,225,96.470207


In [36]:
#
cake_melt[['recipe', 'layer']] = cake_melt['recipe:position'].str.split(':', expand=True)
cake_melt.head()

,recipe:position,variable,value,recipe,layer
0,a:bottom,225,61.738655,a,bottom
1,a:top,225,51.709751,a,top
2,b:bottom,225,57.095320,b,bottom
3,b:top,225,82.455004,b,top
4,c:bottom,225,96.470207,c,bottom


In [46]:
#
cake_tidy = cake_melt.pivot_table(index=['layer', 'variable'],
                      columns=['recipe'],
                      values=['value']).reset_index()
cake_tidy

layer variable      value                                 
recipe                           a          b          c          d
0       bottom      225  61.738655  57.095320  96.470207  52.799753
1       bottom      250  53.912627  61.904369  52.001358  58.670419
2       bottom      275  74.414730  61.196980  92.893227  51.747686
3       bottom      300  98.786784  99.248541  65.473084  56.183110
4          top      225  51.709751  82.455004  71.306308  96.873178
5          top      250  52.009735  95.224151  82.795477  76.101363
6          top      275  68.576858  98.594881  92.098049  59.571620
7          top      300  50.228470  58.169349  53.960273  50.971626

* c. Which recipe, on average, is the best?

In [61]:
for col in cake_tidy.iloc[:, 2:5]:
    print(f'Recipe {col} = {cake_tidy[col].mean()}')

Recipe ('value', 'a') = 63.922201303273
Recipe ('value', 'b') = 76.73607424644436
Recipe ('value', 'c') = 75.87474783650127


In [55]:
cake_tidy.iloc[:, 2:5] # return the columns i need to do my loop

value                      
recipe          a          b          c
0       61.738655  57.095320  96.470207
1       53.912627  61.904369  52.001358
2       74.414730  61.196980  92.893227
3       98.786784  99.248541  65.473084
4       51.709751  82.455004  71.306308
5       52.009735  95.224151  82.795477
6       68.576858  98.594881  92.098049
7       50.228470  58.169349  53.960273

* d. Which oven temperature, on average, produces the best results?

In [63]:
cake_tidy.groupby('variable').max().mean()

/var/folders/qh/kjmdb2zx3n3b2ng_n4q3cyn00000gn/T/ipykernel_45846/1505083730.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  cake_tidy.groupby('variable').max().mean()
/var/folders/qh/kjmdb2zx3n3b2ng_n4q3cyn00000gn/T/ipykernel_45846/1505083730.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cake_tidy.groupby('variable').max().mean()


       recipe
value  a         72.213199
       b         93.880644
       c         84.407999
       d         72.182318
dtype: float64

* e. Which combination of recipe, rack position, and temperature gives the best result?

In [66]:
cake_tidy.max()

          recipe
layer                     top
variable                  300
value     a         98.786784
          b         99.248541
          c         96.470207
          d         96.873178
dtype: object

In [81]:
cake_tidy.max()[3]

99.2485405378462

In [85]:
cake_tidy[(cake_tidy == cake_tidy.max()[3]).any(axis=1)]

layer variable      value                                
recipe                           a          b          c         d
3       bottom      300  98.786784  99.248541  65.473084  56.18311

Bonus: explore the other tables in the tidy_data database and reshape them as necessary so that they are in a tidy format.